In [2]:
import os
from utils import load_secrets

os.environ["HF_TOKEN"] = load_secrets()["HF_TOKEN"]

In [1]:
import spacy

nlp = spacy.load("uk_ner_web_trf_13class")

/opt/anaconda3/envs/UCU_5K2S_Recommender_Systems_Project/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/UCU_5K2S_Recommender_Systems_Project/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/UCU_5K2S_Recommender_Systems_Project/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/UCU_5K2S_Recommender_Systems_Project/lib/python3.10/site-packages/spacy_transformers/laye

In [32]:
import json
import os
import mlflow
import mlflow.spacy
from huggingface_hub import login
from datasets import Dataset


mlflow.set_tracking_uri('http://127.0.0.1:5000')


def ner_inference(text):
    """Perform NER on the input text using spaCy."""
    doc = nlp(text)
    entities = [
        {
            'text': ent.text,
            'label': ent.label_,
            'start_char': ent.start_char,
            'end_char': ent.end_char
        }
        for ent in doc.ents
    ]
    result = {'entities': entities}
    return result

def ner_inference_mlflow(text, experiment_name):
    """Run the NER inference and log inputs and outputs using MLflow."""
    # Set the experiment; create it if it doesn't exist
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run() as run:
        mlflow.log_param("input_text", text)
        try:
            result = ner_inference(text)
            # Log entities as an artifact instead of a parameter
            mlflow.log_dict(result, 'entities.json')
            # Log the model only if it hasn't been logged before
            if not mlflow.active_run().data.tags.get('model_logged'):
                mlflow.spacy.log_model(spacy_model=nlp, artifact_path="uk_ner_model")
                mlflow.set_tag('model_logged', 'true')
            print(f"MLflow run ID: {run.info.run_id}")
        except Exception as e:
            mlflow.log_param('error', str(e))
            print(f"Error during MLflow logging: {e}")
            raise
    return result


def save_run_to_hf(input_text, predicted_entities):
    """Upload the inference data to a Hugging Face Dataset repository."""
    
    token = os.environ.get('HF_TOKEN')
    if not token:
        raise EnvironmentError("HF_TOKEN environment variable not set.")
    login(token=token)
    
    data = {'input_text': [input_text], 'predicted_entities': [predicted_entities]}
    updated_dataset = Dataset.from_dict(data)
    
    dataset_repo_id = 'pofce/test_inference'
    updated_dataset.push_to_hub(dataset_repo_id, token=token)
    print(f"Dataset updated and pushed to the hub: {dataset_repo_id}")


def inference_pipeline(text, experiment_name):
    """Complete inference pipeline: NER inference, logging, and uploading."""
    try:
        result = ner_inference_mlflow(text, experiment_name)
        save_run_to_hf(input_text=text, predicted_entities=result['entities'])
        return json.dumps(result, ensure_ascii=False, indent=1)
    
    except Exception as e:
        print(f"Error during the inference pipeline: {e}")
        raise

if __name__ == "__main__":
    input_text = "Львів — столиця України. Володимир Зеленський є президентом України."
    output = inference_pipeline(input_text, "test_pipeline")

/opt/anaconda3/envs/UCU_5K2S_Recommender_Systems_Project/lib/python3.10/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
2024/11/02 13:34:12 WARNING mlflow.spacy: Generating only the spacy flavor for the provided spacy model. This means the model can be loaded back via `mlflow.spacy.load_model`, but cannot be loaded back using pyfunc APIs like `mlflow.pyfunc.load_model` or via the `mlflow models` CLI commands. MLflow will only generate the pyfunc flavor for spacy models containing a pipeline component that is an instance of spacy.pipeline.TextCategorizer.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the en

MLflow run ID: 14be722e813740eb88cd49ffdf69c5de


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset updated and pushed to the hub: pofce/test_inference
